# Project: Sudoku Solver

Feedback: https://forms.gle/Le3RAsMEcYqEyswEA

Write a program to solve a suduku puzzle.  It should accept a list of 81 values, with '.' representing unknown numbers, and return a completed puzzle.  Print an error message of a solution cannot be found. 

## Sudoku Rules
The objective is to fill a 9x9 grid so that each cell contains a digit from 1 to 9, following these rules:
* Unique in Row: Each row must contain the digits 1 to 9 without any repetition.
* Unique in Column: Each column must also contain the digits 1 to 9 without repetition.
* Unique in 3x3 Sub-grid: The grid is divided into nine 3x3 sub-grids (also known as "boxes" or "regions"). Each sub-grid must contain the digits 1 to 9, with no repeats.

In every Sudoku puzzle:
* Some cells are pre-filled with numbers to give clues and create a unique solution.
* The player can only place numbers in empty cells.
* The puzzle is solved when every cell is filled and all the rules are satisfied.

Additional Points:
* A valid Sudoku puzzle will have only one solution.
* The puzzle’s difficulty depends on the number and placement of given numbers, which influences the logical techniques needed to solve it.

## Strategy
This is a search problem, but we need to use optimizations to reduce the search space so it doesn't take too long to solve each problem!

**Search** - starting at one corner and stepping through cell by cell, place a number, and then check that it is valid.  E.g. if you place a 3, check that there are no threes in the same row, column, and square.  If a conflict is found, try another number.  If none of 1-9 are without conflicts, step back to the previous cell you placed a number in and increment it.  This strategy can take a long time to complete as there are many combinations of numbers to try, but is a path to solving every possible puzzle.  We may need to introduce optimizations to reduce time required to find a solution. 

**Checking cells with fewest number of possibilities first** - compute a list of possible values for all unknown cells and start checking values of cells with the fewest possible values first. 

**Look for naked singles and hidden singles** - A naked single is a cell that only has one possible solution based on the values of cells in it's row, column, and square.  A hidden single is a cell that has multiple possibilities, but one of it's possibilities is not possible for any other cell in its row, column, or square, so it must belong to this cell.

**Set theory optimizations** - E.g. the Phistomefel Ring can be used in addition to the rows, columns, squares checks to ruduce numbers of possibilities for cells. 

### Search Strategies
There are a number of strategies for searching potential solution space for a problem.  

* Uninformed Search - when we are searching blindly for the goal.
  * Bredth First
  * Depth First
  * Uniform Cost Search
* Informed Search - when we have a heuristic to tell is how close a state/node is to the goal.
  * Greedy/Best-First Search
  * A* Search
  * Graph Search

**Bredth First** explores all nodes at the present depth before exploring deeper and is helpful when you're trying to find the shortest path to something, but can be memory intensive.
* A
* B
* C
* D

For this sudoku problem, a bredth first search would be akin to making copies of the puzzle for each possibility for a cell, and then making copies of each of those copies for each possibility for the next cell.  This would result in us having as many as 9^(81-17) copies of the puzzle in memory:
* 81 being the total number of cells
* 17 being the minimum number of cells required to create a valid sudoku puzzle
* 9 being the number of possibilities for each cell

**Depth First** is helpful when we need depth to test/eliminate possibilities and minimizes memory use.  It may not work if the search space is very deep. 
* A
  * AA
    * AAA
    * AAB

A depth first approach would result in having a maximum of only 81-17 copies of the puzzle in memory and works better for sudoku.

**Greedy Search** could be used to explore nodes that are the most constrained first in order to reduce the search space.

**Heuristics for Informed Search**
* Minimum Remaining Values (MRV) Heuristic - solve cells that have the fewest possibilities first
* Degree Heuristic - work on cells that have the most impact on empty cells in the same row/column/sub-grid
* Least Constraining Value (LCV) Heuristic - try values for cells that impose the least restriction on other cells to avoid conflicts.

**Constraint propagation** - whenever a cell is solved, it removes that number as a possibility from all cells in the same row, column, and sub-grid. Constraint propagation is often used in tandem with backtracking, as it can simplify the puzzle and reduce the search space


## Sample Problems
There's a big dataset of sample problems here: https://www.kaggle.com/datasets/radcliffe/3-million-sudoku-puzzles-with-ratings

A sample problem is a list of numbers and '.', with '.' being any value that we need to find. For example:

    ...81.....2........1.9..7...7..25.934.2............5...975.....563.....4......68.



In [19]:
samples = [
    '...81.....2........1.9..7...7..25.934.2............5...975.....563.....4......68.',
    '1..5.37..6.3..8.9......98...1.......8761..........6...........7.8.9.76.47...6.312',
    '..5...74.3..6...19.....1..5...7...2.9....58..7..84......3.9...2.9.4.....8.....1.3',
    '........5.2...9....9..2...373..481.....36....58....4...1...358...42.......978...2',
]
for sample in samples:
    print(sample)
    sample = list(sample)
    # Convert it to a 2D list
    puzzle = [sample[i:i+9] for i in range(0, 81, 9)]
    for row in puzzle:
        print(row)
    print()

...81.....2........1.9..7...7..25.934.2............5...975.....563.....4......68.
['.', '.', '.', '8', '1', '.', '.', '.', '.']
['.', '2', '.', '.', '.', '.', '.', '.', '.']
['.', '1', '.', '9', '.', '.', '7', '.', '.']
['.', '7', '.', '.', '2', '5', '.', '9', '3']
['4', '.', '2', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '5', '.', '.']
['.', '9', '7', '5', '.', '.', '.', '.', '.']
['5', '6', '3', '.', '.', '.', '.', '.', '4']
['.', '.', '.', '.', '.', '.', '6', '8', '.']

1..5.37..6.3..8.9......98...1.......8761..........6...........7.8.9.76.47...6.312
['1', '.', '.', '5', '.', '3', '7', '.', '.']
['6', '.', '3', '.', '.', '8', '.', '9', '.']
['.', '.', '.', '.', '.', '9', '8', '.', '.']
['.', '1', '.', '.', '.', '.', '.', '.', '.']
['8', '7', '6', '1', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '6', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '7']
['.', '8', '.', '9', '.', '7', '6', '.', '4']
['7', '.', '.', '.', '6', '.', '3', '1', '2']

..5...

## Example code using plain python
Using plain python loops and lists, we can step through rows and columns using for loops pretty easily, but things quickly get complex when looking at columns and the squares:

In [20]:
# Example in plain python using loops and lists
for row_num, row in enumerate(puzzle):
    for col_num, cell in enumerate(row):
        if cell != '.':
            continue
        row_set = set(row)
        col_set = set([puzzle[i][col_num] for i in range(9)])
        row_lower, row_upper = row_num//3*3, row_num//3*3+3
        col_lower, col_upper = col_num//3*3, col_num//3*3+3
        square_set = set([puzzle[i][j] for i in range(row_lower, row_upper) for j in range(col_lower, col_upper)])

        print('cell', row_num, col_num, row_set, col_set, square_set)

cell 0 0 {'5', '.'} {'7', '5', '.'} {'2', '9', '.'}
cell 0 1 {'5', '.'} {'.', '9', '8', '1', '3', '2'} {'2', '9', '.'}
cell 0 2 {'5', '.'} {'9', '4', '.'} {'2', '9', '.'}
cell 0 3 {'5', '.'} {'7', '2', '.', '3'} {'9', '2', '.'}
cell 0 4 {'5', '.'} {'6', '.', '8', '4', '2'} {'9', '2', '.'}
cell 0 5 {'5', '.'} {'9', '.', '3', '8'} {'9', '2', '.'}
cell 0 6 {'5', '.'} {'1', '4', '.', '5'} {'5', '.', '3'}
cell 0 7 {'5', '.'} {'.', '8'} {'5', '.', '3'}
cell 1 0 {'2', '9', '.'} {'7', '5', '.'} {'2', '9', '.'}
cell 1 2 {'2', '9', '.'} {'9', '4', '.'} {'2', '9', '.'}
cell 1 3 {'2', '9', '.'} {'7', '2', '.', '3'} {'9', '2', '.'}
cell 1 4 {'2', '9', '.'} {'6', '.', '8', '4', '2'} {'9', '2', '.'}
cell 1 6 {'2', '9', '.'} {'1', '4', '.', '5'} {'5', '.', '3'}
cell 1 7 {'2', '9', '.'} {'.', '8'} {'5', '.', '3'}
cell 1 8 {'2', '9', '.'} {'2', '.', '5', '3'} {'5', '.', '3'}
cell 2 0 {'9', '2', '.', '3'} {'7', '5', '.'} {'2', '9', '.'}
cell 2 2 {'9', '2', '.', '3'} {'9', '4', '.'} {'2', '9', '.'}
cell 2

## Example code using Numpy
For problems like this, numpy has a lot of advantages over regular python.  We can still use loops to step through cells in the puzzle if we want, but many opereations can be vectorized to reduce the need for loops, and numpy array slicing makes it easier to select a row, column, or a square from the array than using plain python lists. 

In [16]:
# Example in numpy using numpy arrays and broadcasting/vectorization
import numpy as np
puzzle = '........5.2...9....9..2...373..481.....36....58....4...1...358...42.......978...2'
puzzle = puzzle.replace('.', '0')
puzzle = np.array(list(puzzle)).astype(np.int8).reshape(9, 9)

for row_num in range(9):
    for col_num in range(9):
        cell = puzzle[row_num, col_num]
        if cell != 0:
            continue
        row_set = np.unique(puzzle[row_num])
        col_set = np.unique(puzzle[:, col_num])
        row_lower, row_upper = row_num//3*3, row_num//3*3+3
        col_lower, col_upper = col_num//3*3, col_num//3*3+3
        square_set = np.unique(puzzle[row_lower:row_upper, col_lower:col_upper])

        print('cell', row_num, col_num, row_set, col_set, square_set)

cell 0 0 [0 5] [0 5 7] [0 2 9]
cell 0 1 [0 5] [0 1 2 3 8 9] [0 2 9]
cell 0 2 [0 5] [0 4 9] [0 2 9]
cell 0 3 [0 5] [0 2 3 7] [0 2 9]
cell 0 4 [0 5] [0 2 4 6 8] [0 2 9]
cell 0 5 [0 5] [0 3 8 9] [0 2 9]
cell 0 6 [0 5] [0 1 4 5] [0 3 5]
cell 0 7 [0 5] [0 8] [0 3 5]
cell 1 0 [0 2 9] [0 5 7] [0 2 9]
cell 1 2 [0 2 9] [0 4 9] [0 2 9]
cell 1 3 [0 2 9] [0 2 3 7] [0 2 9]
cell 1 4 [0 2 9] [0 2 4 6 8] [0 2 9]
cell 1 6 [0 2 9] [0 1 4 5] [0 3 5]
cell 1 7 [0 2 9] [0 8] [0 3 5]
cell 1 8 [0 2 9] [0 2 3 5] [0 3 5]
cell 2 0 [0 2 3 9] [0 5 7] [0 2 9]
cell 2 2 [0 2 3 9] [0 4 9] [0 2 9]
cell 2 3 [0 2 3 9] [0 2 3 7] [0 2 9]
cell 2 5 [0 2 3 9] [0 3 8 9] [0 2 9]
cell 2 6 [0 2 3 9] [0 1 4 5] [0 3 5]
cell 2 7 [0 2 3 9] [0 8] [0 3 5]
cell 3 2 [0 1 3 4 7 8] [0 4 9] [0 3 5 7 8]
cell 3 3 [0 1 3 4 7 8] [0 2 3 7] [0 3 4 6 8]
cell 3 7 [0 1 3 4 7 8] [0 8] [0 1 4]
cell 3 8 [0 1 3 4 7 8] [0 2 3 5] [0 1 4]
cell 4 0 [0 3 6] [0 5 7] [0 3 5 7 8]
cell 4 1 [0 3 6] [0 1 2 3 8 9] [0 3 5 7 8]
cell 4 2 [0 3 6] [0 4 9] [0 3 5 7 8]
ce

In [ ]:
# And if we wanted to get a little more weird with numpy, we could make a 3d array
# having boolean values for possible values of each cell, and a shape of (9, 9, 9):
import numpy as np
puzzle = '........5.2...9....9..2...373..481.....36....58....4...1...358...42.......978...2'
puzzle = puzzle.replace('.', '0')
puzzle = np.array(list(puzzle)).astype(np.int8).reshape(9, 9)
possible_values = np.ones((9, 9, 9), dtype=bool)

for row_num in range(9):
    pass

## Recursion
A side effect of using search, or optimized search, to solve a sudoku puzzle is that we require
recursion to evaluate a changing board while tracking our progress and what has been tried so far. 

### Stack based approach 



### Recursive function approach